In [3]:
import os
from venn import venn
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage,AnnotationBbox,TextArea
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cv2
import itertools

In [4]:
while os.getcwd().split('\\')[-1] != 'ecological-networks':
    %cd ..
from Species_Reintroductions import *

d:\Github\ecological-networks\Code\01_Species_Reintroductions
d:\Github\ecological-networks\Code
d:\Github\ecological-networks


In [5]:
M_Real = UnpickleObj(r"D:\Github\ecological-networks\Code\01_Species_Reintroductions\01_Simulation_Execution\M_Real.pkl")
type(M_Real)

dict

In [65]:
M_df = M_Real['M_PL_041']
X = np.array(M_df)
print(X.shape)
NestednessCalculator(X).nodf(X)

(31, 43)


0.27348225199102394

In [89]:
D = {}
# for NetworkName in tqdm(['M_PL_041']):
for NetworkName in tqdm(M_Real.keys()):

    M_df = M_Real[NetworkName]
    NetM = getNetDefinitions(M_df)
    X = np.array(M_df)

    mat = np.array(M_df)
    # Rows
    po_mat = np.dot(mat, mat.T)
    degrees = mat.sum(axis=1)
    neg_delta = (degrees != degrees[:, np.newaxis])
    deg_matrix = degrees * np.ones_like(po_mat)
    deg_minima = np.minimum(deg_matrix, deg_matrix.T)
    n_pairs_modified = (po_mat / (2. * deg_minima))*neg_delta
    n_pairs_modified[n_pairs_modified == 0] = np.nan
    n_pairs_rows = np.nansum(n_pairs_modified, axis = 0)
    
    # Cols
    po_mat = np.dot(mat.T, mat)
    degrees = mat.sum(axis=0)
    neg_delta = (degrees != degrees[:, np.newaxis])
    deg_matrix = degrees * np.ones_like(po_mat)
    deg_minima = np.minimum(deg_matrix, deg_matrix.T)
    n_pairs_modified = (po_mat / (2. * deg_minima))*neg_delta
    n_pairs_modified[n_pairs_modified == 0] = np.nan
    n_pairs_cols = np.nansum(n_pairs_modified, axis = 0)

    # Total
    n_pairs_minval = np.min([np.min(n_pairs_rows), np.min(n_pairs_cols)])
    n_pairs_maxval = np.max([np.max(n_pairs_rows), np.max(n_pairs_cols)])
    n_pairs_rows = (n_pairs_rows - n_pairs_minval) / (n_pairs_maxval - n_pairs_minval)
    n_pairs_cols = (n_pairs_cols - n_pairs_minval) / (n_pairs_maxval - n_pairs_minval)

    # dict
    N_rows = {list(M_df.index)[i]: n_pairs_rows[i] for i in range(len(n_pairs_rows))}
    N_cols = {list(M_df.columns)[i]: n_pairs_cols[i] for i in range(len(n_pairs_cols))}
    N = {}
    N.update(N_rows)
    N.update(N_cols)

    D[NetworkName] = [M_df.shape, NestednessCalculator(X).nodf(X), N, nx.degree_centrality(NetM), nx.closeness_centrality(NetM), nx.betweenness_centrality(NetM)]

100%|██████████| 30/30 [00:01<00:00, 15.22it/s]


In [ ]:
N = 15
fig, ax = plt.subplots(N,3, figsize = (15, 50), dpi = 100, facecolor = 'none', sharex = 'col', sharey = 'row')
fig.tight_layout(pad = 1)
for row, NetworkName in itertools.islice(enumerate(D.keys()),0,N,1):
    row = row % N

    x_data = np.array(list(D[NetworkName][3].values()))
    y_data = np.array(list(D[NetworkName][5].values()))
    ax[row,0].scatter(x_data, y_data, s = 15, color = plt.cm.viridis(0.5))
    corr = pd.DataFrame({0:x_data, 1:y_data}).corr(method='spearman')[0][1]
    cls = LinearRegression().fit(x_data.reshape(-1,1), y_data)
    ax[row,0].plot([0,0.4], cls.predict(np.array([0,0.4]).reshape(-1,1)), color = plt.cm.viridis(0), linestyle = 'dashed')
    r2 = round(cls.score(x_data.reshape(-1,1), y_data),2)
    ax[row,0].text(0.73,0.05, r'$corr = $' + str(round(corr,2)) + '\n' + r'$R^2 = $' + str(round(r2,2)), 
                            transform=ax[row,0].transAxes, 
                            fontsize = 14, 
                            color = 'blue',
                            va='bottom', ha='left', 
                            bbox = dict(boxstyle='round', facecolor='whitesmoke', alpha=1.0)
                            )

    x_data = np.array(list(D[NetworkName][3].values()))
    y_data = np.array(list(D[NetworkName][4].values()))
    ax[row,1].scatter(x_data, y_data, s = 15, color = plt.cm.viridis(0.5))
    corr = pd.DataFrame({0:x_data, 1:y_data}).corr(method='spearman')[0][1]
    cls = LinearRegression().fit(x_data.reshape(-1,1), y_data)
    ax[row,1].plot([0,0.4], cls.predict(np.array([0,0.4]).reshape(-1,1)), color = plt.cm.viridis(0), linestyle = 'dashed')
    r2 = round(cls.score(x_data.reshape(-1,1), y_data),2)
    ax[row,1].text(0.73,0.05, r'$corr = $' + str(round(corr,2)) + '\n' + r'$R^2 = $' + str(round(r2,2)), 
                            transform=ax[row,1].transAxes, 
                            fontsize = 14, 
                            color = 'blue',
                            va='bottom', ha='left', 
                            bbox = dict(boxstyle='round', facecolor='whitesmoke', alpha=1.0)
                            )
    
    x_data = np.array(list(D[NetworkName][3].values()))
    y_data = np.array(list(D[NetworkName][2].values()))
    ax[row,2].scatter(x_data, y_data, s = 15, color = plt.cm.viridis(0.5))
    corr = pd.DataFrame({0:x_data, 1:y_data}).corr(method='spearman')[0][1]
    cls = LinearRegression().fit(x_data.reshape(-1,1), y_data)
    ax[row,2].plot([0,0.4], cls.predict(np.array([0,0.4]).reshape(-1,1)), color = plt.cm.viridis(0), linestyle = 'dashed')
    r2 = round(cls.score(x_data.reshape(-1,1), y_data),2)
    ax[row,2].text(0.73,0.05, r'$corr = $' + str(round(corr,2)) + '\n' + r'$R^2 = $' + str(round(r2,2)), 
                            transform=ax[row,2].transAxes, 
                            fontsize = 14, 
                            color = 'blue',
                            va='bottom', ha='left', 
                            bbox = dict(boxstyle='round', facecolor='whitesmoke', alpha=1.0)
                            )

    ax[row,0].grid(linestyle = 'dashed', zorder = 0)
    ax[row,1].grid(linestyle = 'dashed', zorder = 0)
    ax[row,2].grid(linestyle = 'dashed', zorder = 0)
    ax[row,0].set_xlim(0,.4)
    ax[row,1].set_xlim(0,.4)
    ax[row,2].set_xlim(0,.4)

    ax[row,0].set_facecolor('none')
    ax[row,1].set_facecolor('none')
    ax[row,2].set_facecolor('none')

    ax[row,0].set_ylabel(NetworkName, fontsize = 16)

ax[row,1].set_xlabel('\nDegree', fontsize = 16)
ax[0,0].set_title('Betweenness', fontsize = 16)
ax[0,1].set_title('Closeness', fontsize = 16)
ax[0,2].set_title('Nestedness', fontsize = 16)

plt.show()

In [143]:
Data = {}
Data['degree'] = []
Data['closeness'] = []
Data['betweenness'] = []
Data['nestedness'] = []
for NetworkName in itertools.islice(M_Real.keys(),0,None,1):
    # 2: N, 3: Degree, 4: Closeness, 5: Betweenness
    n = D[NetworkName][2]
    b = D[NetworkName][5]
    c = D[NetworkName][4]
    d = D[NetworkName][3]
    x = np.array([[d[species],c[species],b[species],n[species]]for species in list(d.keys())])

    Data['degree'] += list(x[:,0])
    Data['closeness'] += list(x[:,1])
    Data['betweenness'] += list(x[:,2])
    Data['nestedness'] += list(x[:,3])

Data['degree'] = np.array(Data['degree'])
Data['closeness'] = np.array(Data['closeness'])
Data['betweenness'] = np.array(Data['betweenness'])
Data['nestedness'] = np.array(Data['nestedness'])

In [144]:
df = pd.DataFrame()
df['Degree'] = Data['degree']
df['Closeness'] = Data['closeness']
df['Betweenness'] = Data['betweenness']
df['Nestedness'] = Data['nestedness']
df

,Degree,Closeness,Betweenness,Nestedness
0,0.250000,0.375000,0.423231,0.294690
1,0.150000,0.379747,0.269380,0.277115
2,0.133333,0.375000,0.215543,0.339236
3,0.116667,0.365854,0.178168,0.351506
4,0.116667,0.333333,0.146541,0.050405
...,...,...,...,...
3150,0.005587,0.295868,0.000000,0.149361
3151,0.005587,0.228608,0.000000,0.074680
3152,0.005587,0.232770,0.000000,0.024893
3153,0.005587,0.268366,0.000000,0.112021


In [216]:
n_iterations = 1000
num_samples = 500
r2_b = []
r2_c = []
r2_n = []
corr_b = []
corr_c = []
corr_n = []
pred_b = []
pred_c = []
pred_n = []
for i in tqdm(range(n_iterations)):
    b_sample = df.sample(num_samples, replace = True)
    d = b_sample['Degree'].values
    c = b_sample['Closeness'].values
    n = b_sample['Nestedness'].values
    b = b_sample['Betweenness'].values

    cls = LinearRegression().fit(d.reshape(-1,1), b)
    r2_b.append(cls.score(d.reshape(-1,1), b))
    corr_b.append(pd.DataFrame({0:d, 1:b}).corr(method='spearman')[0][1])
    pred_b.append(list(cls.predict(np.array([0.01,0.1,0.2,0.3,0.4,0.5]).reshape(-1,1))))

    cls = LinearRegression().fit(d.reshape(-1,1), c)
    r2_c.append(cls.score(d.reshape(-1,1), c))
    corr_c.append(pd.DataFrame({0:d, 1:c}).corr(method='spearman')[0][1])
    pred_c.append(list(cls.predict(np.array([0.01,0.1,0.2,0.3,0.4,0.5]).reshape(-1,1))))

    cls = LinearRegression().fit(d.reshape(-1,1), n)
    r2_n.append(cls.score(d.reshape(-1,1), n))
    corr_n.append(pd.DataFrame({0:d, 1:n}).corr(method='spearman')[0][1])
    pred_n.append(list(cls.predict(np.array([0.01,0.1,0.2,0.3,0.4,0.5]).reshape(-1,1))))

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (15,5), dpi = 600, sharey = True, facecolor = 'none')
ax[0].scatter(Data['degree'], Data['betweenness'], s = 5, color = plt.cm.viridis(0.5))
ax[0].plot([0.01,0.1,0.2,0.3,0.4,0.5], np.mean(pred_b, axis = 0), color = plt.cm.viridis(0), linestyle = 'dashed')
ax[0].text(0.4,0.85, r'$corr = $' + str(round(np.mean(corr_b),2)) + r'$\,\pm\,$' + str(round(np.std(corr_b),3)) + '\n' + r'$R^2 = $' + str(round(np.mean(r2_b),2)) + r'$\,\pm\,$' + str(round(np.std(r2_b),3)),
            transform=ax[0].transAxes,
            fontsize = 14,
            color = 'blue',
            va='bottom', ha='left',
            bbox = dict(boxstyle='round', facecolor='none', alpha=1.0)
            )
ax[0].fill_between([0.01,0.1,0.2,0.3,0.4,0.5], np.percentile(pred_b, 2.5, axis = 0), np.percentile(pred_b, 97.5, axis = 0), color = plt.cm.viridis(0), alpha = 0.2)

ax[1].scatter(Data['degree'], Data['closeness'], s = 5, color = plt.cm.viridis(0.5))
ax[1].plot([0.01,0.1,0.2,0.3,0.4,0.5], np.mean(pred_c, axis = 0), color = plt.cm.viridis(0), linestyle = 'dashed')
ax[1].text(0.4,0.85, r'$corr = $' + str(round(np.mean(corr_c),2)) + r'$\,\pm\,$' + str(round(np.std(corr_c),3)) + '\n' + r'$R^2 = $' + str(round(np.mean(r2_c),2)) + r'$\,\pm\,$' + str(round(np.std(r2_c),3)),
            transform=ax[1].transAxes,
            fontsize = 14,
            color = 'blue',
            va='bottom', ha='left',
            bbox = dict(boxstyle='round', facecolor='none', alpha=1.0)
            )
ax[1].fill_between([0.01,0.1,0.2,0.3,0.4,0.5], np.percentile(pred_c, 2.5, axis = 0), np.percentile(pred_c, 97.5, axis = 0), color = plt.cm.viridis(0), alpha = 0.2)

ax[2].scatter(Data['degree'], Data['nestedness'], s = 5, color = plt.cm.viridis(0.5))
ax[2].plot([0.01,0.1,0.2,0.3,0.4,0.5], np.mean(pred_n, axis = 0), color = plt.cm.viridis(0), linestyle = 'dashed')
ax[2].text(0.4,0.85, r'$corr = $' + str(round(np.mean(corr_n),2)) + r'$\,\pm\,$' + str(round(np.std(corr_n),3)) + '\n' + r'$R^2 = $' + str(round(np.mean(r2_n),2)) + r'$\,\pm\,$' + str(round(np.std(r2_n),3)),
            transform=ax[2].transAxes,
            fontsize = 14,
            color = 'blue',
            va='bottom', ha='left',
            bbox = dict(boxstyle='round', facecolor='none', alpha=1.0)
            )
ax[2].fill_between([0.01,0.1,0.2,0.3,0.4,0.5], np.percentile(pred_n, 2.5, axis = 0), np.percentile(pred_n, 97.5, axis = 0), color = plt.cm.viridis(0), alpha = 0.2)


ax[0].grid(linestyle = 'dashed', zorder = 0)
ax[1].grid(linestyle = 'dashed', zorder = 0)
ax[2].grid(linestyle = 'dashed', zorder = 0)

ax[0].set_xlabel('Degree', fontsize = 14)
ax[1].set_xlabel('Degree', fontsize = 14)
ax[2].set_xlabel('Degree', fontsize = 14)

ax[0].set_ylabel('Betweenness', fontsize = 14)
ax[1].set_ylabel('Closeness', fontsize = 14)
ax[2].set_ylabel('Nestedness', fontsize = 14)

ax[0].set_ylim(0,1.0)

plt.show()